# Sprint10 DeepNeuralNetwork

In [1]:
import numpy as np

# 【問題1】全結合層のクラス化
全結合層のクラス化を行なってください。


以下に雛形を載せました。コンストラクタで重みやバイアスの初期化をして、あとはフォワードとバックワードのメソッドを用意します。重みW、バイアスB、およびフォワード時の入力Xをインスタンス変数として保持しておくことで、煩雑な入出力は不要になります。


なお、インスタンスも引数として渡すことができます。そのため、初期化方法のインスタンスinitializerをコンストラクタで受け取れば、それにより初期化が行われます。渡すインスタンスを変えれば、初期化方法が変えられます。


また、引数として自身のインスタンスselfを渡すこともできます。これを利用してself.optimizer.update(self)という風に層の重みの更新が可能です。更新に必要な値は複数ありますが、すべて全結合層が持つインスタンス変数にすることができます。


初期化方法と最適化手法のクラスについては後述します。

In [74]:
class FC():
    """
    ノード数n_nodes1からn_nodes2への全結合層
    Parameters
    ----------
    n_nodes1 : int
      前の層のノード数
    n_nodes2 : int
      後の層のノード数
    initializer : 初期化方法のインスタンス
    optimizer : 最適化手法のインスタンス
    """
    def __init__(self, input, output, initializer, optimizer):
        self.optimizer = optimizer
        self.input = input
        self.output = output
        self.initializer = initializer

         # 初期化
        # initializerのメソッドを使い、self.Wとself.Bを初期化する       
        self.W = self.initializer.W(self.input, self.output)
        self.B = self.initializer.B(self.output)


    def forward(self, X):
        """
        フォワード
        Parameters
        ----------
        X : 次の形のndarray, shape (batch_size, n_nodes1)
            入力
        Returns
        ----------
        A : 次の形のndarray, shape (batch_size, n_nodes2)
            出力
        """
        return np.dot(X, self.W ) + self.B



    def backward(self, dA):
        """
        バックワード
        Parameters
        ----------
        dA : 次の形のndarray, shape (batch_size, n_nodes2)
            後ろから流れてきた勾配
        Returns
        ----------
        dZ : 次の形のndarray, shape (batch_size, n_nodes1)
            前に流す勾配
        """
        dZ = np.dot(dA, self.W.T)

        # 更新
        self = self.optimizer.update(self, dZ)
        return dZ

# 【問題2】初期化方法のクラス化
初期化を行うコードをクラス化してください。


前述のように、全結合層のコンストラクタに初期化方法のインスタンスを渡せるようにします。以下の雛形に必要なコードを書き加えていってください。標準偏差の値（sigma）はコンストラクタで受け取るようにすることで、全結合層のクラス内にこの値（sigma）を渡さなくてすむようになります。


これまで扱ってきた初期化方法はSimpleInitializerクラスと名付けることにします。

In [67]:
class SimpleInitializer:
    """
    ガウス分布によるシンプルな初期化
    Parameters
    ----------
    sigma : float
      ガウス分布の標準偏差
    """
    def __init__(self, sigma):
        self.sigma = sigma

    def W(self, n_nodes1, n_nodes2, activation='tanh'):
        """
        重みの初期化
        Parameters
        ----------
        n_nodes1 : int
          前の層のノード数
        n_nodes2 : int
          後の層のノード数
        Returns
        ----------
        W :
            個別の重み
        """
        if activation == 'tanh' or 'sigmoid':
            return self.sigma * XavierInitializer().cal(n_nodes1, n_nodes2)
        else:
            return self.sigma * HeInitializer().cal(n_nodes1, n_nodes2)

    def B(self, n_nodes1, activation='tanh'):
        """
        バイアスの初期化
        Parameters
        ----------
        n_nodes2 : int
          後の層のノード数
        Returns
        ----------
        B :
            個別のバイアス
        """
        if activation == 'tanh' or 'sigmoid':
            return np.random.randn(1, n_nodes1) #temp
        else:
            return np.random.randn(1,n_nodes1) #temp

# 【問題3】最適化手法のクラス化
最適化手法のクラス化を行なってください。


最適化手法に関しても初期化方法同様に全結合層にインスタンスとして渡します。バックワードのときにself.optimizer.update(self)のように更新できるようにします。以下の雛形に必要なコードを書き加えていってください。


これまで扱ってきた最適化手法はSGDクラス（Stochastic Gradient Descent、確率的勾配降下法）として作成します。

In [65]:
class SGD():
    """
    確率的勾配降下法
    Parameters
    ----------
    lr : 学習率
    """
    def __init__(self, lr):
        self.lr = lr

    def update(self, layer, dZ):
        """
        ある層の重みやバイアスの更新
        Parameters
        ----------
        layer : 更新前の層のインスタンス
        """
        print(vars(layer))
        layer.W -= self.lr * np.dot(dZ.T, layer.dA)  #back_W3 = np.dot(self.z2.T, back_a3)
        layer.B -= self.lr * np.sum(layer.A, axis=0) # back_b3 = np.sum(back_a3, axis=0)


# 【問題4】活性化関数のクラス化
活性化関数のクラス化を行なってください。


ソフトマックス関数のバックプロパゲーションには交差エントロピー誤差の計算も含む実装を行うことで計算が簡略化されます。

In [22]:
class Softmax:
    '''
    ソフトマックス関数の計算を行う
    '''
    def __init__(self):
        return
    
    def forward(self, A):
        return np.exp(A) / np.sum(np.exp(A), axis=1).reshape(-1, 1) 

    def backward(self, dZ, Y):
        return dZ - Y


class Tanh:
    '''
    ハイパボリックタンジェント関数の計算を行う
    '''
    def __init__(self):
        return
    
    def forward(self, A):
        self.A = np.tanh(A)
        return self.A
    
    def backward(self, dZ):
        return dZ * (1 - np.tanh(self.A) ** 2)

# 【問題5】ReLUクラスの作成
現在一般的に使われている活性化関数であるReLU（Rectified Linear Unit）をReLUクラスとして実装してください。


ReLUは以下の数式です。

$$
f
(
x
)
=
R
e
L
U
(
x
)
=
{
x
if 
x
>
0
,
0
if 
x
≦
0
.
$$

$x$ : ある特徴量。スカラー


実装上はnp.maximumを使い配列に対してまとめて計算が可能です。


numpy.maximum — NumPy v1.15 Manual


一方、バックプロパゲーションのための $x$ に関する $f(x)$ の微分は以下のようになります。

$$
∂
f
(
x
)
∂
x
=
{
1
if 
x
>
0
,
0
if 
x
≦
0
.
$$

数学的には微分可能ではないですが、 $x=0$ のとき $0$ とすることで対応しています。


フォワード時の $x$ の正負により、勾配を逆伝播するかどうかが決まるということになります。

In [6]:
class Relu():
    '''
    ReLu関数の計算を行う
    '''
    def __init__(self):
        pass
    
    def cal(self, X):
        return np.maximum(0, X)


# 【問題6】重みの初期値
ここまでは重みやバイアスの初期値は単純にガウス分布で、標準偏差をハイパーパラメータとして扱ってきました。しかし、どのような値にすると良いかが知られています。シグモイド関数やハイパボリックタンジェント関数のときは Xavierの初期値 （またはGlorotの初期値）、ReLUのときは Heの初期値 が使われます。


XavierInitializerクラスと、HeInitializerクラスを作成してください。

In [7]:
class XavierInitializer:
    '''
    シグモイド関数・ハイパボリックタンジェント関数を使用する際の重みの初期値
    '''
    def __init__(self):
        return
    
    def cal(self, node_input, node_output):
        return np.random.randn(node_input, node_output) / np.sqrt(node_input)
    
    
    
class HeInitializer:
    '''
    ReLU関数を使用する際の重みの初期値
    '''
    def __init__(self):
        pass

    def cal(self, node_input, node_output):
        return np.random.randn(node_input, node_output) / np.sqrt(node_input) * np.sqrt(2)


# 【問題7】最適化手法
学習率は学習過程で変化させていく方法が一般的です。基本的な手法である AdaGrad のクラスを作成してください。


まず、これまで使ってきたSGDを確認します。

$$
W
′
i
=
W
i
−
α
E
(
∂
L
∂
W
i
)
B
′
i
=
B
i
−
α
E
(
∂
L
∂
B
i
)
$$

$\alpha$ : 学習率（層ごとに変えることも可能だが、基本的にはすべて同じとする）


$\frac{\partial L}{\partial W_i}$ : $W_i$ に関する損失 $L$ の勾配


$\frac{\partial L}{\partial B_i}$ : $B_i$ に関する損失 $L$ の勾配


$E()$ : ミニバッチ方向にベクトルの平均を計算


続いて、AdaGradです。バイアスの数式は省略しますが、重みと同様のことをします。


更新された分だけその重みに対する学習率を徐々に下げていきます。イテレーションごとの勾配の二乗和 $H$ を保存しておき、その分だけ学習率を小さくします。


学習率は重み一つひとつに対して異なることになります。

$$
H
′
i
=
H
i
+
E
(
∂
L
∂
W
i
)
×
E
(
∂
L
∂
W
i
)
W
′
i
=
W
i
−
α
1
√
H
′
i
E
(
∂
L
∂
W
i
)
$$

$H_i$ : i層目に関して、前のイテレーションまでの勾配の二乗和（初期値は0）


$H_i^{\prime}$ : 更新した $H_i$

In [8]:
class Adagrad:
    '''
    学習率の最適化
    '''
    def __init__(self, node_num):
        self.lr = lr
        self.h = None

    def update(self, params, grads):
        if self.h is None:
            self.h ={}
            for key, val in params.items():
                self.h[key] = np.zeros_like(val)
        
        for key in params, key():
            self.h[key] += grads[key] * grads[key]
            parms[key] -= self.lr * grads[key] / (np.sqrt( self.h[key] ) + 1e-7 )


# 【問題8】クラスの完成
任意の構成で学習と推定が行えるScratchDeepNeuralNetrowkClassifierクラスを完成させてください。

In [9]:
from keras.datasets import mnist
import numpy as np

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)


from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)


from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train_one_hot = enc.fit_transform(y_train[:, np.newaxis])
y_test_one_hot = enc.transform(y_val[:, np.newaxis])

11501568/11490434 [==============================] - 0s 0us/step


In [10]:
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ
    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 20, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self._X = X[shuffle_index]
        self._y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self._X[p0:p1], self._y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self._X[p0:p1], self._y[p0:p1]

In [75]:
class ScratchDeepNeuralNetrowkClassifier():
    '''
    ベースのクラス
    ---

    '''
    def __init__(self, sigma, lr, n_node1, n_node2, n_output, n_epoch, batch_size, verbose=False):
        self.verbose = verbose
        self.sigma = sigma
        self.lr = lr
        self.n_nodes1 = n_node1
        self.n_nodes2 = n_node2
        self.n_output = n_output
        self.n_epoch = n_epoch
        self.batch_size = batch_size
    

    def fit(self, X, y, X_val=None, y_val=None):
        '''
        NN分類器を学習する
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            訓練データの特徴量
        y : 次の形のndarray, shape (n_samples, )
            訓練データの正解値
        X_val : 次の形のndarray, shape (n_samples, n_features)
            検証データの特徴量
        y_val : 次の形のndarray, shape (n_samples, )
            検証データの正解値
        '''
        self.n_features = X.shape[1]


        optimizer = SGD(self.lr)


        self.FC1 = FC(self.n_features, self.n_nodes1, SimpleInitializer(self.sigma), optimizer)
        self.activation1 = Tanh()
        self.FC2 = FC(self.n_nodes1, self.n_nodes2, SimpleInitializer(self.sigma), optimizer)
        self.activation2 = Tanh()
        self.FC3 = FC(self.n_nodes2, self.n_output, SimpleInitializer(self.sigma), optimizer)
        self.activation3 = Softmax()

        #エポックループ
        for epoch in range(n_epoch):
            #ミニバッチ取得
            get_mini_batch = GetMiniBatch(X, y, batch_size=batch_size)
            #バッチループ
            for mini_X_train, mini_y_train in get_mini_batch:
                # print(vars(self.FC1))

                A1 = self.FC1.forward(mini_X_train)
                Z1 = self.activation1.forward(A1)
                A2 = self.FC2.forward(Z1)
                Z2 = self.activation2.forward(A2)
                A3 = self.FC3.forward(Z2)
                Z3 = self.activation3.forward(A3)
                


                dA3 = self.activation3.backward(Z3, mini_y_train) # 交差エントロピー誤差とソフトマックスを合わせている
                dZ2 = self.FC3.backward(dA3)
                dA2 = self.activation2.backward(dZ2) 
                dZ1 = self.FC2.backward(dA2)
                dA1 = self.activation1.backward(dZ1)
                dZ0 = self.FC1.backward(dA1) # dZ0は使用しない





sigma= 0.01
lr = 0.01
n_nodes1 = 400
n_nodes2 = 200
n_output = 10
n_epoch = 1
batch_size = 100

a = ScratchDeepNeuralNetrowkClassifier(sigma, lr, n_nodes1, n_nodes2, n_output, n_epoch, batch_size)
a.fit(X_train, y_train_one_hot, X_val, y_test_one_hot)

{'optimizer': <__main__.SGD object at 0x7f77fed38c50>, 'input': 200, 'output': 10, 'initializer': <__main__.SimpleInitializer object at 0x7f77ff56cd50>, 'W': array([[ 1.49287142e-04, -8.22676102e-04, -5.58927587e-04, ...,
         2.89543987e-04, -6.73889337e-04,  7.50171396e-04],
       [ 1.02159692e-03, -5.00480662e-04, -1.03430373e-03, ...,
        -4.72541705e-04, -9.32873357e-04,  1.09918454e-03],
       [ 7.07604328e-05,  7.86412460e-04,  2.71168333e-04, ...,
        -5.31464250e-04,  2.11600570e-04, -3.42415805e-04],
       ...,
       [ 7.76552320e-04, -9.39236085e-04,  2.97892722e-04, ...,
         1.22111209e-03,  1.51876731e-03, -3.65515919e-04],
       [-6.55648182e-04, -5.52554131e-04, -1.07037298e-04, ...,
        -8.26049310e-05,  5.83472747e-04, -1.74217941e-04],
       [-5.52890392e-04,  8.60227671e-05,  1.41740182e-03, ...,
         5.30549846e-04, -1.40613215e-04,  8.11609471e-04]]), 'B': array([[ 0.3984179 , -0.48485058,  0.43063656, -1.23565076,  0.87290986,
      

AttributeError: ignored

# 【問題9】学習と推定
層の数や活性化関数を変えたいくつかのネットワークを作成してください。そして、MNISTのデータを学習・推定し、Accuracyを計算してください。